In [851]:
import json as j
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

from nltk.stem import WordNetLemmatizer

In [852]:
data = pd.read_json('resp', orient='coloumns')
data = pd.read_json((data['questions']).to_json(), orient='index')
data.shape

(2000, 12)

In [853]:
remove = ['created_at','creator_id','id','private','question_url','status','updated_at','view_count','category','topic','question_title']
data = data[data.columns.difference(remove)]
data =  data.apply(lambda x: x.astype(str).str.lower())
data

,question_description
0,i loss my hair... how to growth a hair...
1,what is diet recommendations for healthy bones
10,precautions to avoid type 2 diabetes?
100,getting pre masturbation problem how to cure it?
1000,मेरे सर के बाल ञपने आप झड़ जाता है। इसे कैसे रो...
1001,how to gain weight if iam 17 year old
1002,how to solve this problem for headaches
1003,my age is 19years but height is 5'3 so how is ...
1004,panderm plus cream is harmful for face?
1005,my weight is 50 should i loose it how to loose...


In [854]:
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
corpus =[]
for i in range(0,1200):
    review = re.sub('[^a-zA-Z]',' ',data['question_description'][i])
    review = review.split()
    porter = PorterStemmer()
    lemma = WordNetLemmatizer()
    review = [porter.stem(t) for t in review]
    review = [lemma.lemmatize(t) for t in review]
    review = [w for w in review if not w in stopwords]
    review =' '.join(review)
    corpus.append(review)
    print(review)

loss hair growth hair
diet recommend healthi bone
decreas regular stress
lot unwant hair hand lag want remov
loss appetit side effect antidiabet probabl solut thi
diet plan type diabet men
ani treatment regain lost speech attack paralysi
reason yellow teeth care taken get back white teeth
precaut avoid type diabet
manag acn natur way
precaut avoid type diabet
possibl solut dri eye
toothach day get rid pain
diet increas weight
sever back pain get rid back pain
lower intens migrain headach
gain weight natur
get rid cinu problem
best way reliev boredom stress retir
symptom lung cirrhosi
precaut taken menopaus stage
best way keep u hyder summer
reduc stress woman
get rid skin pore
even minut pour ear drop solut tilt head liquid flow ear enter ear canal pool would reason start use becaus wa feel uneasi airtight unabl listen clearli left ear sinc last two day
sever spine problem help
red wine good health advantag disadvantag
p benefit beetroot beetroot juic everyday p
control hair fall woman

In [855]:
y = pd.read_csv("category.csv")

In [856]:
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.333)

In [857]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(0, 1), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=800)),
                     ('clf', SVC(kernel ='linear'))])


model = pipeline.fit(X_train, y_train)

C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [858]:
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

In [859]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

In [860]:
feature_names

array(['aankho', 'aati', 'ab', 'accid', 'accupuntur', 'ach', 'achiev',
       'acid', 'acn', 'acut', 'adhik', 'advantag', 'advic', 'affecr',
       'affect', 'age', 'agn', 'ago', 'airtight', 'alcer', 'alcohol',
       'allergi', 'alot', 'alov', 'alway', 'amentia', 'ani', 'antibiot',
       'antidiabet', 'antioxid', 'anxieti', 'anyon', 'anyth', 'appetit',
       'appl', 'arm', 'arogyavannu', 'arthriti', 'ask', 'astama', 'attack',
       'avoid', 'ayurb', 'ayurved', 'azi', 'baad', 'babi', 'backpain',
       'bacteria', 'bada', 'badli', 'baeta', 'bahut', 'balanc', 'ballanc',
       'banana', 'banti', 'barhega', 'baw', 'beard', 'becom', 'bed',
       'beetroot', 'befor', 'belli', 'benefit', 'best', 'better', 'bhi',
       'bhut', 'bicep', 'big', 'bigger', 'biggest', 'bilirubin', 'black',
       'blede', 'blockapot', 'blood', 'bloold', 'bodi', 'bodybuild',
       'bone', 'boredom', 'bow', 'boy', 'bp', 'brain', 'breakfast',
       'breast', 'breathless', 'bright', 'btay', 'build', 'builder',

In [861]:
print("accuracy score: " + str(model.score(X_test, y_test)))

accuracy score: 0.7475


In [862]:
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test,y_pred)
acc

0.74750000000000005

In [863]:
print(model.predict(['strong']))

['Others']


In [864]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(estimator = model, X = X_train, y = y_train, cv=10)
accuracy

C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

array([ 0.75268817,  0.7       ,  0.68604651,  0.75609756,  0.69620253,
        0.72727273,  0.71052632,  0.71232877,  0.57534247,  0.70422535])